In [2]:
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm import tqdm
import re

# NER Imports
import spacy
import en_core_web_sm
from fuzzywuzzy import fuzz

ModuleNotFoundError: No module named 'spacy'

In [9]:
df = pd.read_csv('./Data/covid19_articles_20201231.csv')

In [10]:
# date to datetime
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369047 entries, 0 to 369046
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   author      181799 non-null  object        
 1   date        369047 non-null  datetime64[ns]
 2   domain      369047 non-null  object        
 3   title       368962 non-null  object        
 4   url         369047 non-null  object        
 5   content     369047 non-null  object        
 6   topic_area  369047 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 19.7+ MB


In [11]:
df

,author,date,domain,title,url,content,topic_area
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business
3,Roberto Torres,2020-01-03,ciodive,"On the road to AI adoption, execs grapple with...",https://www.ciodive.com/news/ai-adoption-execs...,CIOs kicked off 2019 with AI as an item to wat...,tech
4,Alden Wicker,2020-01-06,instyle,Red Carpet Sustainability After Coronavirus Sh...,https://www.instyle.com/fashion/red-carpet-cor...,When the coronavirus pandemic is over and life...,consumer
...,...,...,...,...,...,...,...
369042,Polly Harrison,2020-12-31,thefintechtimes,A Human Touch Will Be a Competitive Edge After...,https://thefintechtimes.com/53867-2/,Niels Pedersen is a Chartered Accountant and S...,finance
369043,NaN,2020-12-31,marketscreener,Datametrex AI : Announces Deploying NexaSecuri...,https://www.marketscreener.com/quote/stock/DAT...,Datametrex AI Limited (TSXV: DM) (FSE: D4G) (O...,business
369044,Polly Harrison,2020-12-31,thefintechtimes,"US Payments: Smart Pension, Episode Six, PAAY ...",https://thefintechtimes.com/us-payments-smart-...,"This December, The Fintech Times is asking ind...",finance
369045,NaN,2020-12-31,marketscreener,"WESTWATER RESOURCES, INC. : Entry into a Mater...",https://www.marketscreener.com/quote/stock/WES...,Item 1.01Entry into a Material Definitive Agre...,business


### NER

In [12]:
ner_processor = en_core_web_sm.load()

#test = ner_processor(df.iloc[0]['content'])
#pprint(set([(X.text, X.label_) for X in test.ents if X.label_ == 'ORG']))

In [17]:
filepath_companies = './Data/company_names.xlsx'
companies_list = pd.read_excel(filepath_companies, sheet_name = 'All', header = None)
companies_list = list(set(sorted(companies_list[0])))

org_stopwords = pd.read_excel(filepath_companies, sheet_name = 'Org_stopwords', header = None)
org_stopwords = list(set(sorted(org_stopwords[0])))
org_stopwords = sorted([word.lower() for word in org_stopwords] + [word.title() for word in org_stopwords])
#org_stopwords

In [18]:
row_idx = 10332
print(df.iloc[row_idx])#['title'])
print(len(df.iloc[row_idx]['content']))

author                                                      NaN
date                                        2020-03-02 00:00:00
domain                                           marketscreener
title         BeiGene : OVERSEAS REGULATORY ANNOUNCEMENT - F...
url           https://www.marketscreener.com/BEIGENE-LTD-261...
content       Hong Kong Exchanges and Clearing Limited and T...
topic_area                                             business
Name: 10332, dtype: object
1116259


In [21]:
# NER Tagging

#df_ner = df[df['date'].between('2020-01-01', '2020-01-31')]
df_ner = df[df['date'].between('2020-01-01', '2020-03-31')]
print('no. of articles: ',len(df_ner))
df_ner = df_ner[df_ner['content'].apply(lambda x: len(x) <= 1000000)]
df_ner = df_ner.reset_index(drop=True)
print('no. of articles after dropping long articles: ', len(df_ner)) # drop articles where text longer than 1,000,000 characters

df_ner['org_names'] = ''

for row in tqdm(range(len(df_ner))):
    pos_tagged = ner_processor(df_ner.iloc[row]['content'])
    org_names = set([X.text for X in pos_tagged.ents if X.label_ == 'ORG'])
    df_ner.at[row,'org_names'] = sorted(org_names)

# Save df after POS tagging
df_ner.to_pickle('./Data/df_ner')
#df_ner.to_csv('./data/df_ner'

  0%|          | 0/48783 [00:00<?, ?it/s]

no. of articles:  48785
no. of articles after dropping long articles:  48783


100%|██████████| 48783/48783 [1:44:09<00:00,  7.81it/s]   


FileNotFoundError: [Errno 2] No such file or directory: './data/df_ner'

In [34]:
# Clean Tagged Names

df_ner = pd.read_pickle('./Data/df_ner')
df_ner_clean = df_ner.copy()

for idx, org_names in tqdm(enumerate(df_ner['org_names'])):
    
    # remove leading and trailing non-letter characters e.g. ' % 2,4 ARCU Aircraft Leasing Limited' -> 'ARCU Aircraft Leasing Limited' Note: this would also drop 3M but companies with number as first letter are very rare
    org_names = set([re.sub('^[^a-zA-Z]*|[^a-zA-Z]*$','',name) for name in org_names])
    
    # remove names that are only 1 or 2 letters
    org_names = set([name for name in org_names if len(name) > 2])
    
    # exclude single word names in org_stopwords list - difference between 2 sets
    org_names = org_names - set(org_stopwords) 
    df_ner_clean.at[idx,'org_names'] = sorted(org_names)
    
# Save df after POS tagging
df_ner_clean.to_pickle('./Data/df_ner_clean')

48783it [00:01, 26676.95it/s]


In [3]:
df_ner_clean = pd.read_pickle('./Data/df_ner_clean')
df_ner_clean

,author,date,domain,title,url,content,topic_area,org_names
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business,"[A Dividend Aristocrat For Capital Gains, Boei..."
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business,"[ADP, CCC, CTAS, Cintas, Cloud-Based Services ..."
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business,"[Credit Suisse, Ford, Musk, NYSE, TSLA, Wedbus..."
3,Roberto Torres,2020-01-03,ciodive,"On the road to AI adoption, execs grapple with...",https://www.ciodive.com/news/ai-adoption-execs...,CIOs kicked off 2019 with AI as an item to wat...,tech,"[CIO, CIO Dive, FedEx, Gartner, Gartner's CIO,..."
4,Alden Wicker,2020-01-06,instyle,Red Carpet Sustainability After Coronavirus Sh...,https://www.instyle.com/fashion/red-carpet-cor...,When the coronavirus pandemic is over and life...,consumer,"[CDC, Chanel, Climate Brief, Moschino, New Sta..."
...,...,...,...,...,...,...,...,...
48778,Jan Lopatka,2020-03-31,reuters,"Czech virus chief defends face masks order, sa...",https://www.reuters.com/article/health-coronav...,- The Czech government’s coronavirus tsar on ...,business,"[Health Ministry, WHO, the World Health Organi..."
48779,Francois Murphy,2020-03-31,reuters,UPDATE 1-Austrian economy to shrink 3.2% in 'm...,https://www.reuters.com/article/austria-econom...,"By Francois Murphy VIENNA, March 31 (Reuters)...",business,"[ECB, Holzmann, ONB, Reuters, The Austrian Nat..."
48780,Francois Murphy,2020-03-31,reuters,Austrian economy to shrink 3.2% in 'moderate' ...,https://www.reuters.com/article/uk-austria-eco...,Austria’s economy will shrink 3.2% this year i...,business,"[ECB, Holzmann, ONB, The Austrian National Ban..."
48781,"Michelle Conlin, Linda So, Brad Heath, Grant S...",2020-03-31,reuters,UPDATE 1-INSIGHT-Coronavirus hits hundreds of ...,https://www.reuters.com/article/health-coronav...,(Updates to include latest available figures f...,business,"[NYPD, Reuters, ShotSpotter, The Dallas Police..."


In [4]:
import itertools 

# List of all org names
ner_org_names = sorted(set(list(itertools.chain(*[names for names in df_ner_clean['org_names']]))))
df_ner_org_names = pd.DataFrame(ner_org_names)
df_ner_org_names.to_pickle('./Data/df_ner_org_names')

In [5]:
len(ner_org_names)

139286

In [7]:
#df_ner[df_ner['org_names'].apply(lambda x: '△167 Appliance' in x)]
df_ner_clean[df_ner_clean['org_names'].apply(lambda x: '△167 Appliance' in x)]

,author,date,domain,title,url,content,topic_area,org_names


In [12]:
#ner_org_names[::-1]
ner_org_names[:50]

['A & PS',
 'A - Measured & Indicated Mineral Resource',
 'A Banking N',
 'A Brown & Co',
 'A Cappella Books',
 'A Cash Flow Machine Constellation Brands',
 'A Centers for Disease Control and Prevention',
 'A Centres for Disease Control and Prevention',
 'A Clarence House',
 'A Comparative Study to Assess the Efficacy, Safety and Immunogenicity of YLB',
 'A Department for International Trade (DIT',
 'A Department of Health and Human Services',
 'A Department of Health and Social Care',
 'A Department of Health and Social care',
 'A Dividend Aristocrat For Capital Gains',
 'A European Space Agency',
 'A European Union',
 'A Final Investment Decision',
 'A Financial Investment Decision',
 'A Fitch',
 'A Food and Drug Administration',
 'A Foreign Private Issuer',
 'A Foreign and Commonwealth Office',
 'A Georgia Department of Health',
 'A Grade AA',
 'A Group',
 'A Historical Perspective',
 'A House of Commons',
 'A Journal of the Plague Year',
 'A K Pranav',
 'A League of Their Own',
 'A

In [47]:
row_idx = 2310
print(df_ner.iloc[row_idx]['org_names'])
df_ner.iloc[row_idx]['content']

['& S Austria Technologie & Systemtechnik AG - First', 'AT&S', 'Ansan', 'Director Communications & Public Relations\n      ', 'EBITDA', 'Group', 'Medical', 'Mobile Devices', 'Nanjangud']


"The spreading of the coronavirus disease is currently impacting the production of AT&S in China. Due to this development, revenue will fall short of expectations in the fourth quarter of the current financial year. AT&S has therefore adjusted its revenue and earnings forecast (previously: revenue at the prior-year level of € 1,028.0 million; EBITDA margin between 20% and 25%) and expects revenue to amount to € 960 million, with an EBITDA margin in the range of 18% to 20%. Subject to staff availability, the plants in Shanghai and Chongqing II will start production in the week of 10 February 2020 after extended New Year's holidays. The Chongqing I plant resumed production at reduced capacity after the New Year's celebrations as planned. AT&S currently considers the following aspects to influence the development in the coming months: a deterioration of the current general conditions, the provision of production materials and personnel, processes in the supply chain and the demand situati

In [63]:
df_jan['filtered_names'] = ''
df_jan['filtered_names_match'] = ''
#for idx, names in tqdm(enumerate(df_jan['org_names'][:50])):
for idx, names in enumerate(df_jan['org_names'][:50]):
    print('\n', idx)
    filtered_names = []
    filtered_names_match = []
    for name in names:
        for list_name in companies_list:
            if len(name.split(' ')) == 1:
                ratio = fuzz.token_set_ratio(name, list_name)
                if ratio >= 90:
                    print('rule1: ', name, '-', list_name, ratio)
                    filtered_names.append(name)
                    filtered_names_match.append(list_name)
            elif set(name.split(' ')).intersection(org_stopwords):
                ratio = fuzz.token_set_ratio(name, list_name)
                if ratio >= 90:
                    print('rule2: ', name, '-', list_name, ratio)
                    filtered_names.append(name)
                    filtered_names_match.append(list_name)
            else:
                ratio = fuzz.partial_ratio(name, list_name)
                if ratio >= 80:
                    print('rule3: ', name, '-', list_name, ratio)
                    filtered_names.append(name)
                    filtered_names_match.append(list_name)
    df_jan.at[idx,'filtered_names'] = set(filtered_names)
    df_jan.at[idx,'filtered_names_match'] = set(filtered_names_match)

/Users/traceyetheridge/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/traceyetheridge/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  



 0
rule1:  Boeing - The Boeing Company 100
rule1:  Boeing - Boeing 100
rule3:  Boeing Boeing - Boeing 100
rule1:  Caterpillar - Caterpillar Inc. 100
rule1:  Caterpillar - Caterpillar 100
rule1:  Cummins - Cummins Inc. 100
rule3:  Cummins Inc - Cummins Inc. 100
rule3:  Cummins, Deere & Company - 3M Company 80
rule3:  Cummins, Deere & Company - Deere & Company 100
rule3:  Deere & Company - 3M Company 80
rule3:  Deere & Company - Deere & Company 100
rule1:  HON - Hon Industries Inc. 100
rule1:  Honeywell - Honeywell International Inc. 100
rule1:  Honeywell - Honeywell 100
rule3:  The Key To Growth Deere & Company - 3M Company 80
rule3:  The Key To Growth Deere & Company - Deere & Company 100
rule3:  The Move Cummins Inc - Cummins Inc. 96

 1
rule1:  Cintas - Cintas Corp. 100
rule3:  Manpower, Inc - Manpower Inc. 92

 2
rule3:  Credit Suisse - Credit Suisse 100
rule1:  Ford - Ford Motor Co 100

 3

 4

 5

 6

 7

 8

 9
rule3:  KB Homes - KB Home 100
rule3:  KB Homes (KBH - KB Home 100
r

In [51]:
df_jan

,author,date,domain,title,url,content,topic_area,org_names,filtered_names,filtered_names_match
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business,"[BA, Boeing, Boeing Boeing, Caterpillar, Cummi...","{Boeing Boeing, Cummins, The Move Cummins Inc,...","{The Boeing Company, Cummins Inc., Honeywell I..."
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business,"[ADP, CAGR, CTAS, Cintas, Cloud-Based Services...","{Manpower, Inc, Cintas}","{Cintas Corp., Manpower Inc.}"
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business,"[Credit Suisse, Ford, Mach-E, Musk, NYSE, TSLA...","{Ford, Credit Suisse}","{Ford Motor Co, Credit Suisse}"
3,Roberto Torres,2020-01-03,ciodive,"On the road to AI adoption, execs grapple with...",https://www.ciodive.com/news/ai-adoption-execs...,CIOs kicked off 2019 with AI as an item to wat...,tech,"[AI, CIO, CIO Dive, CIOs, FedEx, Gartner, Gart...",{},{}
4,Alden Wicker,2020-01-06,instyle,Red Carpet Sustainability After Coronavirus Sh...,https://www.instyle.com/fashion/red-carpet-cor...,When the coronavirus pandemic is over and life...,consumer,"[BAFTA, CDC, COVID-19, Chanel, Climate Brief, ...",{},{}
...,...,...,...,...,...,...,...,...,...,...
1933,By Denise Grady and Roni Caryn Rabin,2020-01-31,nytimes,195 Quarantined in California After Fleeing Co...,https://www.nytimes.com/2020/01/31/health/quar...,The United States government has imposed a two...,business,"[Emory University, Science Times, Vanderbilt U...",,
1934,Karissa Bell,2020-01-31,mashable,Facebook will remove some coronavirus conspira...,https://mashable.com/article/facebook-will-rem...,Facebook is cracking down on coronavirus rumor...,tech,"[Facebook, Google]",,
1935,By Isabella Kwai,2020-01-31,nytimes,An Outbreak of Racist Sentiment as Coronavirus...,https://www.nytimes.com/2020/01/31/world/austr...,The Australia Letter is a weekly newsletter fr...,business,"[British Airways, Coronavirus Outbreak Tests W...",,
1936,By Zolan Kanno-Youngs,2020-01-31,nytimes,Trump Administration Adds Six Countries to Tra...,https://www.nytimes.com/2020/01/31/us/politics...,WASHINGTON — President Trump on Friday added s...,business,"[Border Protection, Boundless Immigration, Cus...",,
